In [173]:
from keras.models import load_model
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
from glob import glob
import generator as gen
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
import tensorflow as tf
#from keras.applications.resnet50 import ResNet50
from keras.applications.resnet import ResNet50

def dice_coeff(y_true, y_pred, smooth=100):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)

    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    return (2 * intersection + smooth) / (union + smooth)

def dice_coeff_loss(y_true, y_pred, smooth=100):
    return 1 - dice_coeff(y_true, y_pred, smooth)

def dice_metric(y_true, y_pred, smooth=100):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)

    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    return (2 * intersection + smooth) / (union + smooth)

def dice_loss(y_true, y_pred, smooth=100):
    return 1 - dice_coeff(y_true, y_pred, smooth)

In [174]:
data_path_test = "G:\\Harsh_Birva_minor_Proj\\FIVES\\test"

batch_size = 1

In [175]:
def DICE_COE(mask1, mask2):
    intersect = np.sum(mask1*mask2)
    fsum = np.sum(mask1)
    ssum = np.sum(mask2)
    dice = (2 * intersect ) / (fsum + ssum)
    dice = np.mean(dice)
    dice = round(dice, 3)
    return dice    

In [176]:
mynet_64 = tf.keras.models.load_model("G:\\Harsh_Birva_minor_Proj\\New_Fives_models\\mynet32_256_16_3branch.keras", custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})

In [177]:
retina_files_test = glob(pathname = data_path_test+'\\Original\\*A*')
mask_files_test = [path.replace('Original', 'Ground truth') for path in retina_files_test]

df_test_A = pd.DataFrame( data = {
    'retina': retina_files_test,
    'mask': mask_files_test
})

In [178]:
retina_files_test = glob(pathname = data_path_test+'\\Original\\*D*')
mask_files_test = [path.replace('Original', 'Ground truth') for path in retina_files_test]

df_test_D = pd.DataFrame( data = {
    'retina': retina_files_test,
    'mask': mask_files_test
})

In [179]:
retina_files_test = glob(pathname = data_path_test+'\\Original\\*_G*')
mask_files_test = [path.replace('Original', 'Ground truth') for path in retina_files_test]

df_test_G = pd.DataFrame( data = {
    'retina': retina_files_test,
    'mask': mask_files_test
})

In [180]:
retina_files_test = glob(pathname = data_path_test+'\\Original\\*_N*')
mask_files_test = [path.replace('Original', 'Ground truth') for path in retina_files_test]

df_test_N = pd.DataFrame( data = {
    'retina': retina_files_test,
    'mask': mask_files_test
})

In [181]:
image_size = 256

In [182]:
def load_and_preprocess_image(file_path):
    img = cv2.imread(file_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (image_size,image_size))
    img_scaled = img_resized.astype(np.float32) / 255.0
    return img_scaled

In [183]:
def load_and_preprocess_mask(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE )
    img = img[:,:,np.newaxis]
    img_resized = cv2.resize(img, (256,256))
    img_scaled = img_resized.astype(np.float32) / 255.0
    img_scaled[img_scaled >= 0.25] = 1
    img_scaled[img_scaled < 0.25] = 0
    return img_scaled

In [184]:
kernel = np.array([[0,1,0],[1,1,1],[0,1,0]],dtype =np.uint8)

# Age-Related

In [185]:
age_dice = []
age_thin_dice = []
age_thick_dice = []

for i in range(50):
    retina_img = load_and_preprocess_image(df_test_A.iloc[i]['retina'])
    mask_img = load_and_preprocess_mask(df_test_A.iloc[i]['mask'])
    
    retina_img = retina_img[np.newaxis,:,:,:]
    pred_img = mynet_64.predict(retina_img)
    pred_img = (np.squeeze(pred_img) > 0.5).astype(np.uint8)
    mask_img = np.squeeze(mask_img).astype(np.uint8)
    
    age_dice.append(DICE_COE(mask_img,pred_img))
    original = mask_img.copy()
    original_pred = pred_img.copy()
    
    pred_img = pred_img.astype(float)
    mask_img = mask_img.astype(float)
    
    for i in range(5):
        mask_img = cv2.erode(mask_img, kernel)
        pred_img = cv2.erode(pred_img, kernel)
        
        mask_img = cv2.dilate(mask_img, kernel)
        pred_img = cv2.dilate(pred_img, kernel)
    
    pred_img = pred_img.astype(np.uint8)
    mask_img = mask_img.astype(np.uint8)
    
    original_thin = cv2.subtract(original,mask_img).astype(np.uint8)
    predicted_thin = cv2.subtract(original_pred,pred_img).astype(np.uint8)
    
    age_thin_dice.append(DICE_COE(original_thin,predicted_thin))
    age_thick_dice.append(DICE_COE(mask_img, pred_img))

1/1 [==============================] - 0s 30ms/step


In [186]:
len(age_dice)

50

In [187]:
print(f'Age Related Dice: {round(np.mean(age_dice),3)}')

Age Related Dice: 0.871


In [188]:
print(f'Age Related Thin Dice: {round(np.mean(age_thin_dice),3)}')

Age Related Thin Dice: 0.726


In [189]:
print(f'Age Related Thick Dice: {round(np.mean(age_thick_dice),3)}')

Age Related Thick Dice: 0.82


# Diabetes

In [190]:
diabetes_dice = []
diabetes_thin_dice = []
diabetes_thick_dice = []

for i in range(50):
    retina_img = load_and_preprocess_image(df_test_D.iloc[i]['retina'])
    mask_img = load_and_preprocess_mask(df_test_D.iloc[i]['mask'])
    
    retina_img = retina_img[np.newaxis,:,:,:]
    pred_img = mynet_64.predict(retina_img)
    pred_img = (np.squeeze(pred_img) > 0.5).astype(np.uint8)
    mask_img = np.squeeze(mask_img).astype(np.uint8)
    
    diabetes_dice.append(DICE_COE(mask_img,pred_img))
    
    original = mask_img.copy()
    original_pred = pred_img.copy()
    
    pred_img = pred_img.astype(float)
    mask_img = mask_img.astype(float)
    
    for i in range(5):
        mask_img = cv2.erode(mask_img, kernel)
        pred_img = cv2.erode(pred_img, kernel)
        
        mask_img = cv2.dilate(mask_img, kernel)
        pred_img = cv2.dilate(pred_img, kernel)
    
    pred_img = pred_img.astype(np.uint8)
    mask_img = mask_img.astype(np.uint8)
    
    original_thin = cv2.subtract(original,mask_img).astype(np.uint8)
    predicted_thin = cv2.subtract(original_pred,pred_img).astype(np.uint8)
    
    diabetes_thin_dice.append(DICE_COE(original_thin,predicted_thin))
    diabetes_thick_dice.append(DICE_COE(mask_img, pred_img))

1/1 [==============================] - 0s 20ms/step


In [191]:
len(diabetes_dice)

50

In [192]:
print(f'Diabetes Dice: {round(np.mean(diabetes_dice),3)}')

Diabetes Dice: 0.841


In [193]:
print(f'Diabetes Thin Dice: {round(np.mean(diabetes_thin_dice),3)}')

Diabetes Thin Dice: 0.653


In [194]:
print(f'Diabetes Thick Dice: {round(np.mean(diabetes_thick_dice),3)}')

Diabetes Thick Dice: 0.793


# Glucoma

In [195]:
glucoma_dice = []
glucoma_thin_dice = []
glucoma_thick_dice = []

for i in range(50):
    retina_img = load_and_preprocess_image(df_test_G.iloc[i]['retina'])
    mask_img = load_and_preprocess_mask(df_test_G.iloc[i]['mask'])
    
    retina_img = retina_img[np.newaxis,:,:,:]
    pred_img = mynet_64.predict(retina_img)
    pred_img = (np.squeeze(pred_img) > 0.5).astype(np.uint8)
    mask_img = np.squeeze(mask_img).astype(np.uint8)
    
    glucoma_dice.append(DICE_COE(mask_img,pred_img))
    
    original = mask_img.copy()
    original_pred = pred_img.copy()
    
    pred_img = pred_img.astype(float)
    mask_img = mask_img.astype(float)
    
    for i in range(5):
        mask_img = cv2.erode(mask_img, kernel)
        pred_img = cv2.erode(pred_img, kernel)
        
        mask_img = cv2.dilate(mask_img, kernel)
        pred_img = cv2.dilate(pred_img, kernel)
    
    pred_img = pred_img.astype(np.uint8)
    mask_img = mask_img.astype(np.uint8)
    
    original_thin = cv2.subtract(original,mask_img).astype(np.uint8)
    predicted_thin = cv2.subtract(original_pred,pred_img).astype(np.uint8)
    
    glucoma_thin_dice.append(DICE_COE(original_thin,predicted_thin))
    glucoma_thick_dice.append(DICE_COE(mask_img, pred_img))

1/1 [==============================] - 0s 30ms/step


In [196]:
len(glucoma_dice)

50

In [197]:
print(f'Glucoma Dice: {round(np.mean(glucoma_dice),3)}')

Glucoma Dice: 0.801


In [198]:
print(f'Glucoma Thin Dice: {round(np.mean(glucoma_thin_dice),3)}')

Glucoma Thin Dice: 0.629


In [199]:
print(f'Glucoma Thick Dice: {round(np.mean(glucoma_thick_dice),3)}')

Glucoma Thick Dice: 0.801


# Normal

In [200]:
normal_dice = []
normal_thin_dice = []
normal_thick_dice = []

for i in range(50):
    retina_img = load_and_preprocess_image(df_test_N.iloc[i]['retina'])
    mask_img = load_and_preprocess_mask(df_test_N.iloc[i]['mask'])
    
    retina_img = retina_img[np.newaxis,:,:,:]
    pred_img = mynet_64.predict(retina_img)
    pred_img = (np.squeeze(pred_img) > 0.5).astype(np.uint8)
    mask_img = np.squeeze(mask_img).astype(np.uint8)
    
    normal_dice.append(DICE_COE(mask_img,pred_img))
    
    original = mask_img.copy()
    original_pred = pred_img.copy()
    
    pred_img = pred_img.astype(float)
    mask_img = mask_img.astype(float)
    
    for i in range(5):
        mask_img = cv2.erode(mask_img, kernel)
        pred_img = cv2.erode(pred_img, kernel)
        
        mask_img = cv2.dilate(mask_img, kernel)
        pred_img = cv2.dilate(pred_img, kernel)
    
    pred_img = pred_img.astype(np.uint8)
    mask_img = mask_img.astype(np.uint8)
    
    original_thin = cv2.subtract(original,mask_img).astype(np.uint8)
    predicted_thin = cv2.subtract(original_pred,pred_img).astype(np.uint8)
    
    normal_thin_dice.append(DICE_COE(original_thin,predicted_thin))
    normal_thick_dice.append(DICE_COE(mask_img, pred_img))

1/1 [==============================] - 0s 30ms/step


In [201]:
len(normal_dice)

50

In [202]:
print(f'Normal Dice: {round(np.mean(normal_dice),3)}')

Normal Dice: 0.861


In [203]:
print(f'Normal Thin Dice: {round(np.mean(normal_thin_dice),3)}')

Normal Thin Dice: 0.704


In [204]:
print(f'Normal Thick Dice: {round(np.mean(normal_thick_dice),3)}')

Normal Thick Dice: 0.825
